## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'E:/Seminararbeit/Code/A1/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [3]:
####################################################################################
###################### 2-1. nonrecursive model by store ############################
####################################################################################

In [4]:
cvs = ['private']
STORES = ['TX_3', 'WI_1', 'WI_2', 'WI_3'] #'CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 

In [5]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [6]:
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
FIRST_DAY = 710
remove_feature = ['id',
                  'state_id',
                  'store_id',
#                   'item_id',
#                   'dept_id',
#                   'cat_id',
                  'date','wm_yr_wk','d','sales']

cat_var = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
cat_var = list(set(cat_var) - set(remove_feature))

In [8]:
grid2_colnm = ['sell_price', 'price_max', 'price_min', 'price_std',
               'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
               'price_momentum', 'price_momentum_m', 'price_momentum_y']

grid3_colnm = ['event_name_1', 'event_type_1', 'event_name_2',
               'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m',
               'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end']

lag_colnm = [ 'sales_lag_28', 'sales_lag_29', 'sales_lag_30',
             'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34',
             'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38',
             'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42',
             
             'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14',
             'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60',
             'rolling_std_60', 'rolling_mean_180', 'rolling_std_180']

mean_enc_colnm = [
    
    'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std', 
    'enc_item_id_state_id_mean', 'enc_item_id_state_id_std',


]

In [9]:
########################### Make grid
#################################################################################
def prepare_data(store):
    
    grid_1 = pd.read_pickle(processed_data_dir+"processedgrid_part_1.pkl")
    grid_2 = pd.read_pickle(processed_data_dir+"processedgrid_part_2.pkl")[grid2_colnm]
    grid_3 = pd.read_pickle(processed_data_dir+"processedgrid_part_3.pkl")[grid3_colnm]

    grid_df = pd.concat([grid_1, grid_2, grid_3], axis=1)
    del grid_1, grid_2, grid_3; gc.collect()
    
    grid_df = grid_df[grid_df['store_id'] == store]
    grid_df = grid_df[grid_df['d'] >= FIRST_DAY]
    
    lag = pd.read_pickle(processed_data_dir+"processedlags_df_28.pkl")[lag_colnm]
    
    lag = lag[lag.index.isin(grid_df.index)]
    
    grid_df = pd.concat([grid_df,
                     lag],
                    axis=1)
    
    del lag; gc.collect()
    

    mean_enc = pd.read_pickle(processed_data_dir+"processedmean_encoding_df.pkl")[mean_enc_colnm]
    mean_enc = mean_enc[mean_enc.index.isin(grid_df.index)]
    
    grid_df = pd.concat([grid_df,
                         mean_enc],
                        axis=1)    
    del mean_enc; gc.collect()
    
    grid_df = reduce_mem_usage(grid_df)
    
    
    
    return grid_df

In [10]:
validation = {
    'cv1' : [1551, 1610],
    'cv2' : [1829,1857],
    'cv3' : [1857, 1885],
    'cv4' : [1885,1913],
    'public' : [1913, 1941],
    'private' : [1941, 1969]
}

### cv1 : 2015-04-28 ~ 2015-06-26

### cv2 : 2016-02-01 ~ 2016-02-28

### cv3 : 2016-02-29 ~ 2016-03-27

### cv4 : 2016-03-28 ~ 2016-04-24

In [11]:
########################### Model params
#################################################################################
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': 1,
                    'seed' : 1995,
                    'n_jobs': 1
                } 

In [12]:
########################### Train Models
#################################################################################

rmsse_bycv = dict()

for cv in cvs:
    print('cv : day', validation[cv])

    pred_list = []
    for store in STORES:

        print(store, 'start')
        grid_df = prepare_data(store)

        model_var = grid_df.columns[~grid_df.columns.isin(remove_feature)]

        tr_mask = (grid_df['d'] <= validation[cv][0]) & (grid_df['d'] >= FIRST_DAY)
        vl_mask = (grid_df['d'] > validation[cv][0]) & (grid_df['d'] <= validation[cv][1])

        train_data = lgb.Dataset(grid_df[tr_mask][model_var], 
                       label=grid_df[tr_mask]['sales'])

        valid_data = lgb.Dataset(grid_df[vl_mask][model_var], 
                           label=grid_df[vl_mask]['sales'])

        m_lgb = lgb.train(lgb_params, train_data, valid_sets = [valid_data, train_data], callbacks=[lgb.log_evaluation(1, True)]) 
        display(pd.DataFrame({'name':m_lgb.feature_name(),
                              'imp':m_lgb.feature_importance()}).sort_values('imp',ascending=False).head(25))
        
        model_name = model_dir+'non_recur_model_'+store+'.bin'
        pickle.dump(m_lgb, open(model_name, 'wb'))
        
        del grid_df, train_data, valid_data, m_lgb, tr_mask, vl_mask; gc.collect

cv : day [1941, 1969]
TX_3 start
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.963127 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6955
[LightGBM] [Info] Number of data points in the train set: 3454993, number of used features: 58
[1]	training's rmse: 3.84083	valid_0's rmse: 0.999955
[2]	training's rmse: 3.82752	valid_0's rmse: 1.00047
[3]	training's rmse: 3.8137	valid_0's rmse: 1.00149
[4]	training's rmse: 3.79943	valid_0's rmse: 1.00328
[5]	training's rmse: 3.78483	valid_0's rmse: 1.00545
[6]	training's rmse: 3.76981	valid_0's rmse: 1.00816
[7]	training's rmse: 3.75427	valid_0's rmse: 1.01155
[8]	traini

,name,imp
0,item_id,167890
15,event_name_1,37459
23,tm_w,32651
52,rolling_mean_180,24178
13,price_momentum_m,21916
53,rolling_std_180,21535
50,rolling_mean_60,20750
51,rolling_std_60,19438
48,rolling_mean_30,17165
49,rolling_std_30,16947


WI_1 start
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.515002 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6363
[LightGBM] [Info] Number of data points in the train set: 3473381, number of used features: 58
[1]	training's rmse: 2.64347	valid_0's rmse: 1.00128
[2]	training's rmse: 2.63196	valid_0's rmse: 1.0026
[3]	training's rmse: 2.62027	valid_0's rmse: 1.00429
[4]	training's rmse: 2.60854	valid_0's rmse: 1.00662
[5]	training's rmse: 2.59699	valid_0's rmse: 1.00928
[6]	training's rmse: 2.58526	valid_0's rmse: 1.01214
[7]	training's rmse: 2.57336	valid_0's rmse: 1.01553
[8]	training's rmse: 2.56106	vali

,name,imp
0,item_id,174401
15,event_name_1,41646
23,tm_w,33966
52,rolling_mean_180,23332
53,rolling_std_180,21628
50,rolling_mean_60,20519
13,price_momentum_m,20203
51,rolling_std_60,19057
48,rolling_mean_30,17372
22,tm_d,16502


WI_2 start
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.065494 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6953
[LightGBM] [Info] Number of data points in the train set: 3472946, number of used features: 58
[1]	training's rmse: 4.41886	valid_0's rmse: 1.0034
[2]	training's rmse: 4.40393	valid_0's rmse: 1.00704
[3]	training's rmse: 4.38846	valid_0's rmse: 1.01163
[4]	training's rmse: 4.37246	valid_0's rmse: 1.01715
[5]	training's rmse: 4.35624	valid_0's rmse: 1.02382
[6]	training's rmse: 4.33977	valid_0's rmse: 1.03102
[7]	training's rmse: 4.32274	valid_0's rmse: 1.0394
[8]	training's rmse: 4.30507	valid

,name,imp
0,item_id,172914
15,event_name_1,31171
23,tm_w,30055
52,rolling_mean_180,20959
50,rolling_mean_60,19208
13,price_momentum_m,18003
53,rolling_std_180,17264
48,rolling_mean_30,16852
22,tm_d,15316
51,rolling_std_60,15127


WI_3 start
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.953338 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6940
[LightGBM] [Info] Number of data points in the train set: 3467725, number of used features: 58
[1]	training's rmse: 3.78618	valid_0's rmse: 0.999423
[2]	training's rmse: 3.77327	valid_0's rmse: 0.999239
[3]	training's rmse: 3.75986	valid_0's rmse: 0.999601
[4]	training's rmse: 3.746	valid_0's rmse: 1.00071
[5]	training's rmse: 3.73187	valid_0's rmse: 1.00248
[6]	training's rmse: 3.71738	valid_0's rmse: 1.00469
[7]	training's rmse: 3.70238	valid_0's rmse: 1.00773
[8]	training's rmse: 3.68668	va

,name,imp
0,item_id,180044
15,event_name_1,37602
23,tm_w,32189
52,rolling_mean_180,22455
13,price_momentum_m,19740
50,rolling_mean_60,19660
53,rolling_std_180,19375
51,rolling_std_60,17991
48,rolling_mean_30,17119
49,rolling_std_30,16384
